In [1]:
ascad_path = '/content/drive/MyDrive/ASCAD_databases/ASCAD.h5'

In [ ]:
import h5py
import numpy as np

def load_ascad_fixed_key(file_path, profiling=True):
  if profiling:
    trace_path='Profiling'
  else:
    trace_path='Attack'

  with h5py.File(file_path, "r") as f:
    X_profiling = np.array(f[f'{trace_path}_traces/traces'])
    Y_profiling = np.array(f[f'{trace_path}_traces/labels'])

    metadata = f[f'{trace_path}_traces/metadata']
    plaintext = np.array(metadata['plaintext'])
    key = np.array(metadata['key'])

    return X_profiling, Y_profiling, plaintext, key

In [ ]:
X_profiling, Y_profiling, plaintext, key = load_ascad_fixed_key(ascad_path, profiling=True)

In [ ]:
X_attack, Y_attack, plaintext_attack, key_attack = load_ascad_fixed_key(ascad_path, profiling=False)

In [ ]:
# Transformer with sin/cos positional encodings

import numpy as np
import tensorflow as tf
# from tensorflow import keras
from tensorflow.keras.layers import Layer

class PositionalEncoding(Layer):
  def __init__(self, d_model, max_len=1000):
    super(PositionalEncoding, self).__init__()
    pos = np.arange(max_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

  def call(self, inputs):
    seq_len = tf.shape(inputs)[1]
    return inputs + self.pos_encoding[:, :seq_len, :]

In [ ]:
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Dense, Dropout, Input, Add

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
  attn = MultiHeadAttention(key_dim=head_size, num_heads=num_heads)(inputs, inputs)
  attn = Dropout(dropout)(attn)

  out1 = LayerNormalization(epsilon=1e-6)(inputs + attn)

  ffn = Dense(ff_dim, activation='relu')(out1)
  ffn = Dense(inputs.shape[-1])(ffn)
  ffn = Dropout(dropout)(ffn)
  return LayerNormalization(epsilon=1e-6)(out1 + ffn)

In [ ]:
!pip install

In [ ]:
from tensorflow.keras.models import Model
from keras._tf_keras.keras.layers import GlobalAveragePooling1D, Dense

def build_transformer(input_shape, num_classes=256, depth=2, head_size=64, num_heads=4, ff_dim=128, dropout=0.2):
  inputs = Input(shape=input_shape)
  print(f"input_shape: {input_shape}")
  x = PositionalEncoding(d_model=input_shape[1])(inputs)

  for _ in range(depth):
    x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

  x = GlobalAveragePooling1D()(x)
  x = Dense(128, activation='relu')(x)
  x = Dropout(dropout)(x)
  outputs = Dense(num_classes, activation='softmax')(x)
  print(f"output_shape: {outputs.shape}")

  return Model(inputs, outputs)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(Y_profiling, num_classes=256)

In [ ]:
X_profiling_reshaped = X_profiling.reshape((-1, 700, 1))

In [ ]:
Y_profiling_sparse = Y_profiling.astype(np.int32)

In [ ]:
model = build_transformer(input_shape=(700,1))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

input_shape: (700, 1)
output_shape: (None, 256)


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 700, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, 700, 1)    │          0 │ input_layer_10[0… │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 700, 1)    │      1,793 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ positional_encod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_64          │ (None, 700, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_36 (Add)        │ (None, 700, 1)    │          0 │ positional_encod… │
│                     │                   │            │ dropout_64[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 700, 1)    │          2 │ add_36[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_54 (Dense)    │ (None, 700, 128)  │        256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_55 (Dense)    │ (None, 700, 1)    │        129 │ dense_54[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_65          │ (None, 700, 1)    │          0 │ dense_55[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_37 (Add)        │ (None, 700, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_65[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 700, 1)    │          2 │ add_37[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 700, 1)    │      1,793 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_67          │ (None, 700, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_38 (Add)        │ (None, 700, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_67[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 700, 1)    │          2 │ add_38[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_56 (Dense)    │ (None, 700, 128)  │        256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_57 (Dense)    │ (None, 700, 1)    │        129 │ dense_56[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_68          │ (None, 700, 1)    │          0 │ dense_57[0][0]    │
│ (Dropout)           │                   │            │                 

 Total params: 37,644 (147.05 KB)

 Trainable params: 37,644 (147.05 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_profiling_reshaped, Y_profiling_sparse, batch_size=128, epochs=50, validation_split=0.1, verbose=1)

Epoch 1/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 7534s 21s/step - accuracy: 0.0042 - loss: 5.5451 - val_accuracy: 0.0044 - val_loss: 5.5450
Epoch 2/50
200/352 ━━━━━━━━━━━━━━━━━━━━ 51:45 20s/step - accuracy: 0.0052 - loss: 5.5441

In [ ]:
model.save('/content/drive/MyDrive/TransformerPoC.keras')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])